In [1]:
!pip install pypdf

## Removing pages

In [45]:
len(pages)

324

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "test.pdf"
)
pages=[]
loader = PyPDFLoader(file_path)
pages_raw = loader.load_and_split()
pages_to_delete = [2,3, 4, 5,6,7,8,9,10,11,12,13,297,298,299,300,301,302,303,304,305,306,307]
for i in pages_raw:
    if i.metadata['page'] not in pages_to_delete:
       pages.append({"page_number": i.metadata['page'],  # adjust page numbers since our PDF starts on page 42
                                "page_char_count": len(i.page_content),
                                "page_word_count": len(i.page_content.split(" ")),
                                "page_sentence_count_raw": len(i.page_content.split(". ")),
                                "page_token_count": len(i.page_content) / 4,  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                                "text": i.page_content}) 

In [3]:
# real text starts from page 14 slice it if too much mnoise
import pandas as pd

In [4]:
print(pages[2])

{'page_number': 16, 'page_char_count': 1423, 'page_word_count': 238, 'page_sentence_count_raw': 4, 'page_token_count': 355.75, 'text': 'Introduction      3Part\xa0 3: The Workouts  includes two chapters designed to help you set goals, includ -\ning and a 13-week program you can modify to your ends.\nPart\xa0 4: Calisthenics for Special Circumstances  includes four chapters covering \nnine-minute workouts, how to work out when you’re pregnant or have limited \nmobility, and also how to help your kids get excited about fitness.\nPart\xa0 5: The Part of Tens  includes two chapters that give you a buffet of tips to tone \nand tighten and how to be successful, as well as a chapter that debunks the top ten \nbodyweight training myths.\nIcons Used in This Book\nIn the margins of almost every page of this book, you find icons, which are there \nto alert you to different types of information. Here’s what they mean:\nThis icon saves you time and energy by showing you a helpful method for doing \

In [5]:
import pandas as pd

df = pd.DataFrame(pages)

In [6]:
from spacy.lang.en import English # see https://spacy.io/usage for install instructions

nlp = English()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/ 
nlp.add_pipe("sentencizer")


In [7]:
type(pages[17]["text"])

str

In [8]:
def listToString(s):
 
    # initialize an empty string
    str1 = ""
 
    # traverse in the string
    for ele in s:
        str1 += ele
 
    # return string
    return str1
# Driver code

In [9]:
from tqdm.auto import tqdm
for item in tqdm(pages):
    item["sentences"] = list(nlp(listToString(item["text"])).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    # Count the sentences 
    item["page_sentence_count_spacy"] = len(item["sentences"])
    
    

  0%|          | 0/293 [00:00<?, ?it/s]

In [10]:
df = pd.DataFrame(pages)

In [11]:
from tqdm.auto import tqdm
num_sentence_chunk_size = 10 

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/293 [00:00<?, ?it/s]

In [12]:
df = pd.DataFrame(pages)

In [13]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

  0%|          | 0/293 [00:00<?, ?it/s]

531

In [14]:
pages_and_chunks[17]

{'page_number': 25,
 'sentence_chunk': '12   PART 1 Total Fitness: No LimitsGetting Comfortable with \nBodyweight\xa0Exercises\nTo beginners, it can feel pretty weird trying to exercise without the typical exer-\ncise machines. Instead, you’re going to start using your body the way it was meant \nto be used\xa0— the way you used it when you were first learning to crawl, walk, run, \nand climb.\nUse muscles you didn’t know you had\nWith the proper bodyweight movements, you can isolate and work all your \nmuscles. Everything from your neck to toes are fair game and tied together. You’ll \ndiscover muscles you have and muscles that might that have atrophied in our \nmodern sedentary lifestyles. You will literally become a stronger person.\nGet off the bench\nYour arms were made to push and pull your bodyweight, not to grasp handles on \nmachines or metal bars while you sit on a bench. I mean, lying down on a soft \nsurface is just fine\xa0— when you’re sleeping. Sitting on a cushioned sea

In [16]:

from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

model = SentenceTransformer(
    "jinaai/jina-embeddings-v2-base-en", # switch to en/zh for English or Chinese
    trust_remote_code=True
)

# control your input sequence length up to 8192
model.max_seq_length = 3000
for item in tqdm(pages_and_chunks):
    item["embedding"] = model.encode(item["sentence_chunk"])


/home/percy/miniconda3/envs/raghub/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


  0%|          | 0/531 [00:00<?, ?it/s]

In [26]:
model.save('/home/percy/Desktop/RAG/')

In [129]:
xx=pd.DataFrame(pages_and_chunks)


In [130]:
xx.to_csv('/home/percy/Desktop/RAG/save_embedding.csv')

In [1]:
import torch
import numpy as np 
import pandas as pd


device = "cuda" if torch.cuda.is_available() else "cpu"

# Import texts and embedding df
text_chunks_and_embedding_df = pd.read_csv("save_embedding.csv")

# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
# text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

In [14]:
!nvidia-smi

Sat Jun 22 09:59:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        Off | 00000000:01:00.0  On |                  N/A |
|  0%   49C    P8              19W / 170W |   9538MiB / 12288MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
text_chunks_and_embedding_df

,Unnamed: 0,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,0,14,Introduction 1IntroductionW elcome to Calist...,1058,177,264.50,[-1.35593578e-01 -2.72922397e-01 2.89607733e-...
1,1,14,The most important takeaway from this book is ...,1750,285,437.50,[-1.20129004e-01 -1.94902048e-01 2.80463904e-...
2,2,14,Introduction 3Part 3: The Workouts includes ...,1586,266,396.50,[-3.71636182e-01 -6.91351742e-02 7.20650777e-...
3,3,14,It’s important to note that virtually everyone...,1488,236,372.00,[-2.06898283e-02 -7.00647235e-02 5.61529338e-...
4,4,14,And they’re for people of all abilities. They ...,984,168,246.00,[-2.97168940e-01 -1.03575937e-01 5.18190742e-...
...,...,...,...,...,...,...,...
100201,100201,322,This is using leverage to increase the exercis...,761,135,190.25,[-1.47650197e-01 -3.34271729e-01 3.74935389e-...
100202,100202,322,You have total control of the resistance. Aero...,2532,332,633.00,[-8.78950730e-02 -6.61716461e-02 5.66305779e-...
100203,100203,322,"Seepulling exercisesbilateral deficit, 288bloo...",21719,2381,5429.75,[-0.15278333 0.00791218 0.04112889 0.177799...
100204,100204,322,"He is the author of many books, including the ...",1373,214,343.25,[-1.49986506e-01 -2.00150952e-01 4.69057381e-...


In [3]:
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

In [4]:
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([100206, 768])

In [5]:
def encode_query(query):
    from sentence_transformers import SentenceTransformer
    from sentence_transformers.util import cos_sim

    model = SentenceTransformer(
        "jinaai/jina-embeddings-v2-base-en", # switch to en/zh for English or Chinese
        trust_remote_code=True
    )
    query_tensor=model.encode(query,convert_to_tensor=True)
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return query_tensor

In [6]:
from sentence_transformers import util, SentenceTransformer

/home/percy/miniconda3/envs/raghub/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# def top_results(encoded_tensor,embeddings):
    # sim=util.dot_score(encoded_tensor,embeddings)[0]

In [7]:
import gc
def print_wrapped(text, wrap_length=80):
    import textwrap
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)
def similarity(encoded_tensor,embeddings):
    sim=util.dot_score(encoded_tensor,embeddings)[0]
    sim= torch.unique(sim)
    scores,indices= torch.topk(sim, k=5)
    return scores,indices
#Loop through zipped together scores and indicies from torch.topk
encoded_tensor=encode_query("Squat")
torch.cuda.empty_cache()
print(encoded_tensor)

/home/percy/miniconda3/envs/raghub/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tensor([-0.0383, -0.3721,  0.1730,  0.5424,  0.0414,  0.2273,  0.5683, -0.1312,
         0.7266,  0.1346, -0.2843, -0.3908, -0.9235,  0.1539, -0.3418,  0.5402,
        -0.1161,  0.1935, -0.1021, -0.2609, -0.6866, -0.1532, -0.6287, -0.2507,
        -0.1353,  0.6821,  0.7366,  0.1955,  0.2304,  1.0647,  0.4901, -0.7249,
        -0.4145,  0.6341,  0.1759, -0.7568, -0.1616, -0.3712,  0.6152,  0.6676,
         0.3230,  0.4903, -0.6086,  0.9239, -0.4998,  0.2547,  0.2513, -0.1362,
        -0.2452, -0.3023,  0.0881, -0.2441, -0.3793, -0.5821,  0.5441, -0.0871,
         0.7379,  0.4406, -0.5265,  0.4211, -0.4091,  0.7719, -0.0439,  0.8862,
         0.7708,  0.3452,  0.0462, -1.1207, -0.0025, -0.0712, -0.0388,  0.1479,
        -0.0849, -0.1689,  0.1684,  1.0847, -0.7428, -0.5378, -0.6297,  0.2713,
         0.6040,  0.2718,  0.7497, -0.3353, -0.8383, -0.4306, -1.1529,  0.2350,
         0.6201,  0.4858, -0.3056,  1.2264, -0.3576,  0.7490,  0.1652,  0.1902,
         0.3553, -0.3467, -0.5110, -0.36

In [8]:
scores,indices=similarity(encoded_tensor,embeddings)
context_items = [pages_and_chunks[i] for i in indices]
print(scores)
context_items

tensor([176.7346, 172.3243, 170.8529, 170.7814, 170.4523], device='cuda:0')


[{'Unnamed: 0': 341,
  'page_number': 14,
  'sentence_chunk': 'Her organizational skills, attention to detail, and absolute dedication are invaluable to his organization’s success. Lastly, we would like to thank the Dummies team for this rare opportunity. It is an honor to become one of your authors and subject matter experts. The relentless drive and firm guidance of our editors Kezia Endsley and Jennifer Yee did not go unnoticed, and it is in the end greatly appreciated!Thank you. Publisher’s AcknowledgmentsAcquisitions Editor: Jennifer YeeManaging Editor: Murari Mukundan/Kristie PylesProject Editor: Kezia EndsleyTechnical Editor: Lou SchulerProduction Editor: Pradesh KumarCover Image: © skynesher/Getty ImagesWILEY END USER LICENSE AGREEMENTGo to www.wiley.com/go/eula to access Wiley’s ebook EULA.',
  'chunk_char_count': 747,
  'chunk_word_count': 102,
  'chunk_token_count': 186.75,
  'embedding': array([-1.27610415e-01,  1.65308379e-02,  7.72747695e-01, -4.76270020e-02,
          9.

In [9]:
def prompt_formatter(query: str, 
                     context_items: list[dict]) -> str:
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])
    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""

    # Update base prompt with context items and query   
    base_prompt = base_prompt.format(context=context, query=query)

    # Create prompt template for instruction-tuned model
    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]
    prompt=dialogue_template
    # Apply the chat template
    #prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          # tokenize=False,
                                          # add_generation_prompt=True)
    return prompt
prompt=prompt_formatter("Squat",context_items)
    

In [10]:
print(prompt)
for i in prompt:
    x=i

[{'role': 'user', 'content': "Based on the following context items, please answer the query.\nGive yourself room to think by extracting relevant passages from the context before answering the query.\nDon't return the thinking, only return the answer.\nMake sure your answers are as explanatory as possible.\nUse the following examples as reference for the ideal answer style.\n\nExample 1:\nQuery: What are the fat-soluble vitamins?\nAnswer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.\n\nExample 2:\nQuery: What are the causes of type 2 diabetes?\nAnswer: Type 2 diabet

In [12]:
from langchain_community.llms import Ollama

In [18]:
del model
gc.collect()
torch.cuda.empty_cache()

NameError: name 'model' is not defined